In [19]:
from src.optimisers.hadamard_gate_reduction import Hadamard_gate_reduction
from src.optimisers.single_qubit_gate_cancellation import Single_qubit_gate_cancellation
from src.optimisers.phase_polynomial_optimizer import optimize_phase_polynomial

from src.converters.netlist_converters import qCircuit_to_netlist, netlist_to_qCircuit
from src.converters.graph_converter import create_graph_from_netlist, create_netlist_from_graph
from src.converters.phase_polynomial_converter import create_phase_polynomial_from_netlist, create_circuit_from_phase_polynomial

from src.utils.graphs import plot_graph
from mpqp import QCircuit
from mpqp.gates import X, CNOT, Z, H , Rz

In [20]:
circ = QCircuit(2)
    
circ.add(Rz(1, 1))
circ.add(CNOT(0, 1))
circ.add(Rz(2, 1))
circ.add(CNOT(0, 1)) 
circ.add(Rz(3, 0))
circ.add(Rz(4, 1))
circ.add(CNOT(1, 0))

print(circ)

                                 ┌───────┐┌───┐
q_0: ───────────■─────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌─┴─┐┌───────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(1) ├┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(4) ├──■──
     └───────┘└───┘└───────┘└───┘└───────┘     


In [21]:
netlist = qCircuit_to_netlist(circ)
print(netlist)
polynome = create_phase_polynomial_from_netlist(netlist)

[Rz([1]), CNOT(·[0], [1]), Rz([1]), CNOT(·[0], [1]), Rz([0]), Rz([1]), CNOT(·[1], [0])]


In [22]:
opti_polynome =  optimize_phase_polynomial(polynome)

In [23]:
netlist2 = create_circuit_from_phase_polynomial(opti_polynome)
print(netlist2)


[CNOT(·[0], [1]), Rz([1]), CNOT(·[0], [1]), Rz([1]), Rz([1]), Rz([0]), CNOT(·[1], [0])]


In [24]:
circ2 = netlist_to_qCircuit(netlist2)
print("before :")
print(circ)
print("after :")
print(circ2)

before :
                                 ┌───────┐┌───┐
q_0: ───────────■─────────────■──┤ Rz(3) ├┤ X ├
     ┌───────┐┌─┴─┐┌───────┐┌─┴─┐├───────┤└─┬─┘
q_1: ┤ Rz(1) ├┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(4) ├──■──
     └───────┘└───┘└───────┘└───┘└───────┘     
after :
                        ┌───────┐         ┌───┐
q_0: ──■─────────────■──┤ Rz(3) ├─────────┤ X ├
     ┌─┴─┐┌───────┐┌─┴─┐├───────┤┌───────┐└─┬─┘
q_1: ┤ X ├┤ Rz(2) ├┤ X ├┤ Rz(4) ├┤ Rz(1) ├──■──
     └───┘└───────┘└───┘└───────┘└───────┘     
